CODES TO ACCESS AND PULL AGSI DATA THEN PRODUCE VISUALIZATIONS

In [ ]:
! pip install matplotlib

In [ ]:
import requests

def fetch_gas_storage_data(country, start_date, end_date, page=1, reverse=False, size=30):
    url = "https://agsi.gie.eu/api"
    params = {
        "country": country,
        "from": start_date,
        "to": end_date,
        "page": page,
        "reverse": reverse,
        "size": size
    }
    headers = {
        "Authorization": f"Bearer {api_key}"
    }
    response = requests.get(url, params=params,headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print("Failed to fetch data:", response.status_code)
        return None

# Example usage
api_key = "ef180adc60bce455847056789a5c08c9"
country_code = "DE"
start_date = "2022-01-01"
end_date = "2022-12-31"
gas_storage_data = fetch_gas_storage_data(country_code, start_date, end_date)


In [ ]:
# Check if data is returned
if gas_storage_data:
    print("Data retrieved successfully!")
    print(gas_storage_data)  # Print or inspect the data here
else:
    print("No data retrieved.")

In [ ]:
import matplotlib.pyplot as plt
from datetime import datetime

# Extracting gas days and gas in storage values
gas_days = [entry['gasDayStart'] for entry in gas_storage_data['data']]
gas_in_storage = [float(entry['gasInStorage']) for entry in gas_storage_data['data']]

# Convert gas days to datetime objects
gas_days = [datetime.strptime(day, '%Y-%m-%d') for day in gas_days]

# Create the plot
plt.figure(figsize=(10, 6))
plt.plot(gas_days, gas_in_storage, marker='o', linestyle='-')
plt.title('Gas Storage Inventory in Germany (2022)')
plt.xlabel('Date')
plt.ylabel('Gas in Storage (millions of cubic meters)')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import requests

def fetch_gas_storage_data(country, start_date, end_date, page=1, reverse=False, size=300):
    url = "https://agsi.gie.eu/api"
    params = {
        "country": country,
        "from": start_date,
        "to": end_date,
        "page": page,
        "reverse": reverse,
        "size": size
    }
    headers = {
        "Authorization": f"Bearer {api_key}"
    }
    all_data = []  # List to store all retrieved data
    while True:
        response = requests.get(url, params=params, headers=headers)
        if response.status_code == 200:
            data = response.json()
            all_data.extend(data['data'])
            if page >= data['last_page']:
                break  # Break the loop if all pages have been retrieved
            page += 1  # Move to the next page
            params['page'] = page  # Update the page parameter for the next request
        else:
            print("Failed to fetch data:", response.status_code)
            return None
    return all_data


In [ ]:
api_key = "ef180adc60bce455847056789a5c08c9"
# Define a list of country codes
country_codes = ["DE", "FR", "PL", "ES", "IT", "HU", "RO", "BG"]

# Define the date range
start_date = "2000-01-01"
end_date = "2023-12-31"

# Initialize an empty dictionary to store gas storage data for each country
gas_storage_data_all_countries = {}

# Loop over each country code and fetch data
for country_code in country_codes:
    gas_storage_data = fetch_gas_storage_data(country_code, start_date, end_date)
    if gas_storage_data:
        gas_storage_data_all_countries[country_code] = gas_storage_data

# gas_storage_data_all_countries will now contain gas storage data for all specified countries


In [ ]:
# Check if data is returned
if gas_storage_data_all_countries:
    print("Data retrieved successfully!")
    print(gas_storage_data_all_countries)  # Print or inspect the data here
else:
    print("No data retrieved.")

1 FIGURE FOR MANY COUNTRIES OVER TIME

In [ ]:
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd

# Define the desired date range
start_date = "2013-01-01"
end_date = "2023-12-31"

# Define a list of country codes
country_codes = ["DE", "FR", "PL", "ES", "IT", "HU", "RO", "BG"]

# Create a single figure
plt.figure(figsize=(10, 6))

# Loop over each country code
for country_code in country_codes:
    # Extract gas storage data for the current country code
    gas_storage_data_country = gas_storage_data_all_countries.get(country_code)
    if gas_storage_data_country:
        # Filter data within the desired date range
        filtered_data = [entry for entry in gas_storage_data_country if start_date <= entry['gasDayStart'] <= end_date]

        # Sort filtered data by date
        filtered_data.sort(key=lambda x: x['gasDayStart'])

        # Extract gas days, gas in storage, and full % values for the current country
        gas_days = [entry['gasDayStart'] for entry in filtered_data]
        gas_in_storage = [float(entry['gasInStorage']) for entry in filtered_data]

        # Convert gas days to datetime objects
        gas_days = [datetime.strptime(day, '%Y-%m-%d') for day in gas_days]

        # Create a DataFrame with the data
        df = pd.DataFrame({
            'gasDayStart': gas_days,
            'gasInStorage': gas_in_storage
        })

        # Set 'gasDayStart' as the index
        df.set_index('gasDayStart', inplace=True)

        # Resample the data to fill in missing dates and interpolate missing values
        df_resampled = df.resample('D').interpolate()

        # Plot the data with label
        plt.plot(df_resampled.index, df_resampled['gasInStorage'], linestyle='-', label=country_code)

plt.legend()
# Customize the plot
plt.title(f'Gas storage ({start_date} to {end_date})')
plt.xlabel('Date')
plt.ylabel('Gas storage (in mln cubic meters)')
plt.xticks(rotation=45)
plt.grid(True)
plt.ylim(bottom=0)
plt.tight_layout()
plt.show()


MULTIPLE FIGURES : 1 PER VARIABLE FOR MANY COUNTRIES OVER TIME

In [ ]:
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd
import numpy as np

# Define the desired date range
start_date = "2013-01-01"
end_date = "2023-12-31"

# Define a list of country codes
country_codes = ["DE", "FR", "PL", "ES", "IT", "HU", "RO", "BG"]

# Define the window size for the moving average filter
window_size = 7

# Loop over each variable
for variable in ['gasInStorage', 'full', 'injection', 'withdrawal']:
    # Create a single figure
    plt.figure(figsize=(10, 6))

    # Loop over each country code
    for country_code in country_codes:
        # Extract gas storage data for the current country code
        gas_storage_data_country = gas_storage_data_all_countries.get(country_code)
        if gas_storage_data_country:
            # Filter data within the desired date range
            filtered_data = [entry for entry in gas_storage_data_country if start_date <= entry['gasDayStart'] <= end_date]

            # Sort filtered data by date
            filtered_data.sort(key=lambda x: x['gasDayStart'])

            # Extract gas days and variable values for the current country
            gas_days = [entry['gasDayStart'] for entry in filtered_data]
            variable_values = [float(entry[variable]) for entry in filtered_data]

            # Convert gas days to datetime objects
            gas_days = [datetime.strptime(day, '%Y-%m-%d') for day in gas_days]

            # Create a DataFrame with the data
            df = pd.DataFrame({
                'gasDayStart': gas_days,
                variable: variable_values
            })

            # Set 'gasDayStart' as the index
            df.set_index('gasDayStart', inplace=True)

            # Resample the data to fill in missing dates and interpolate missing values
            df_resampled = df.resample('D').interpolate()

            # Apply moving average filter for 'injection' and 'withdrawal' variables
            if variable in ['injection', 'withdrawal']:
                df_resampled[variable] = df_resampled[variable].rolling(window=window_size, center=True, min_periods=1).mean()

            # Plot the data with label
            plt.plot(df_resampled.index, df_resampled[variable], linestyle='-', label=country_code)

    plt.legend()
    # Customize the plot
    plt.title(f'{variable.capitalize()} ({start_date} to {end_date})')
    plt.xlabel('Date')
    if variable == 'gasInStorage':
        plt.ylabel('Gas storage (in mln cubic meters)')
    elif variable == 'full':
        plt.ylabel('Full percentage (%)')
    elif variable in ['injection', 'withdrawal']:
        plt.ylabel('Daily volume (in mln cubic meters)')
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.ylim(bottom=0)
    plt.tight_layout()

    # Add a note for 'injection' and 'withdrawal' graphs
    if variable in ['injection', 'withdrawal']:
        font_dict = {'family': 'sans-serif', 'style': 'italic', 'size': 10, 'color': 'grey'}
        plt.text(0.02, 0.93, f'Note: {window_size}-day moving average applied', transform=plt.gca().transAxes, va='top', fontdict=font_dict)
    plt.show()


In [ ]:
import pandas as pd

# Assuming gas_storage_data is the data you pulled
data = gas_storage_data['data']

# Create a DataFrame from the data
df = pd.DataFrame(data)

# Display the DataFrame
print(df)
